### 사전학습된 모델 fine tuning하기 
사전 학습된 모델을 사용하면 상당한 이점이 있다.    
사전 학습된 모델을 사용하는 경우, 자신의 task와 관련된 데이터셋을 사용해서 학습한다.     
허깅페이스를 사용해서 사전학습된 모델을 파인튜닝해보자     

In [1]:
from datasets import load_dataset
dataset = load_dataset("codyburker/yelp_review_sampled")

In [2]:
dataset["train"][100]

{'text': "One of our favorite places in the short north! It's a cozy place but have always been able to find a spot. It's a little dark at night which may bother some people but it's just fine with me! I love their calamari appetizer, but beware it's a little spicy! Good beer selection! I have yet to get any of their cocktails but definitely plan on trying some they sound great. You should totally give this place a try!",
 'date': '8/4/2015 5:10',
 'stars': 4}

In [3]:
#리뷰에 관한 텍스트 데이터가 담겨있다.
#데이터 전처리를 해보자
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_func(examples):
    return tokenizer(examples["text"],padding = "max_length",truncation=True)

tokenized = dataset.map(tokenize_func,batch_size=True)

텍스트를 처리하기 위해서 서로다른 길이의 텍스트들을 패딩으로 맞춰주었고, 데이터셋을 한번에 처리하기 위해서 dataset map 메서드를 사용해서 한번에 적용해주었다.    
필요한 경우 fine tuning을 위해서 데이터셋의 작은 부분 집함을 만들어서 시간을 줄일 수 있다.     

```python
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
```     

와 같이 사용할 수 있다.

먼저 transformers에서 모델을 가져오고 나의 task에 사용할 레이블 수를 정한다.    
yelp review dataset의 경우 5개의 레이블이다.

In [4]:
#train

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


다음으로 정할 수 있는 모든 하이퍼 파라미터와 다양한 훈련 옵션을 활성화한다.    
이를 위해서 TrainingArguments 클래스를 생성한다.     
튜토리얼에서 기본 훈련 하이퍼 파라미터로 시작하지만, 나중에 내가 자유롭게 실험해서 나에게 맞는 최적의 설정을 찾을 수 있다.    
훈련에서 체크포인트를 저장할 위치를 지정한다. 

In [5]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir='test_trainer')

trainer는 훈련 중에 모델 성능을 자동으로 평가하지 않는다.    
평가지표를 계산하고 보고할 함수를 Trainer에 전달해야한다.    
evaluate 라이브러리는 간단한 accuracy함수를 제공한다.    

In [6]:
import evaluate

metric = evaluate.load("accuracy")

In [7]:
import numpy as np
def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions,references=labels)

fine tuning중 평가 지표를 모니터링하려면 훈련인수에 evaluation_strategy 파라미터를 지정해주면 된다.

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir='test_trainer',evaluation_strategy='epoch')

In [9]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'
#tokenized2 = tokenized.rename_column("stars","labels")

#small_train_dataset = tokenized2["train"].shuffle(seed=42).select(range(1000))

#small_eval_dataset = tokenized2["test"].shuffle(seed=42).select(range(1000))
model


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
dataset2 = dataset.rename_column('stars','labels')
tokenized2 = dataset2.map(tokenize_func,batch_size=True)


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [12]:
small_train_dataset = tokenized2["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized2["test"].shuffle(seed=42).select(range(1000))

trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)


c:\Users\HOME\anaconda3\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [13]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
